In [153]:
from time import time

import string
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression


In [154]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

def clean(listoftweets):
    
    listoffiltered=[]
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for i in range(0,len(listoftweets)):
         
         
         listoftweets[i]=listoftweets[i].replace('@USER', '') #Remove mentions (@USER)
         
         listoftweets[i]=listoftweets[i].replace('URL', '')
         listoftweets[i]=listoftweets[i].replace('&amp', 'and')
         listoftweets[i]=listoftweets[i].replace('&lt', '')
         listoftweets[i]=listoftweets[i].replace('&gt', '')
         listoftweets[i]=listoftweets[i].replace('\d+', '')
         listoftweets[i] = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          (listoftweets[i])) 
         listoftweets[i]=emoji_pattern.sub(r'', listoftweets[i])
         listoftweets[i]=listoftweets[i].translate(str.maketrans('', '', string.punctuation))
         listoftweets[i]=listoftweets[i].strip()
         listoftweets[i] = re.sub(' +', ' ', listoftweets[i])
         listoftweets[i]=listoftweets[i].lower()
         word_tokens = word_tokenize(listoftweets[i])
         filteredsen = [w for w in word_tokens if w not in stopwords and w != ' ']
         listoffiltered.append(filteredsen)
    return listoffiltered
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [155]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', min_df=4)

 


In [156]:
def train_LR_model(path_to_train_file):
   
   df=pd.read_csv(path_to_train_file,sep='\t')
   label = df['subtask_a']
   columns = ['id','subtask_b','subtask_c']
   df.drop(columns, axis=1, inplace=True)
   listoftweets = []
   for i in range(0,len(df)):
     listoftweets.append(df.iloc[i,0])
   clean(listoftweets)
   # creating bag of words on train data
   jk=vectorizer.fit_transform(listoftweets)
   logreg = LogisticRegression()
   logreg.fit(jk, label)
   return logreg


In [157]:
lm = train_LR_model('olid-training-v1.0.tsv')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [158]:
def test_LR_model(path_to_test_file, LR_model):
   df1=pd.read_csv(path_to_test_file ,sep='\t')
   df2=pd.read_csv(path_to_test_file,sep='\t')
   columns=['id']
   df1.drop(columns,axis=1,inplace=True)
   listoftweetstest = []
   for i in range(0,len(df1)):
     listoftweetstest.append(df1.iloc[i,0])
   clean(listoftweetstest) 
   labeltest = pd.read_csv('labels-levela.csv', header=None)
   labeltest1 = labeltest.iloc[:,1]
   # creating bag of words for test data
   vector = vectorizer.transform(listoftweetstest)
   # making it dense
   vector =vector.toarray()
   # predictions
   testpredictions = LR_model.predict(vector)
   print(testpredictions)
   # probabilities
   yu = LR_model.predict_proba(vector)
   prob=[]
   for bn in yu:
     prob.append(bn[1])
   count=0;
   # accuracy calculation
   for i in range(0,len(testpredictions)):
     if testpredictions[i]==labeltest1[i]:
            count=count+1
   accuracy = count/len(testpredictions)    
   print("accuracy",accuracy)  
  # df2.insert(2,'predictions', testpredictions )
  # df2.insert(3,'prob', prob)   
  # adding columns in original data frame
   df2['predictions']=testpredictions
   df2['probabilities'] = prob
   print(df2)
   df2.to_csv('LR.csv', sep='\t',index=False)


In [159]:
test_LR_model('testset-levela.tsv', lm)

['OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF'
 'NOT' 'NOT' 'OFF' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'NOT'
 'NOT' 'OFF' 'NOT' 'OFF' 'NOT' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT'
 'NOT' 'NOT' 'OFF' 'OFF' 'OFF' 'OFF' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF'
 'OFF' 'NOT' 'NOT' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'OFF'
 'OFF' 'NOT' 'NOT' 'OFF' 'OFF' 'NOT' 'NOT' 'OFF' 'NOT' 'NOT' 'OFF' 'NOT'
 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'NOT' 'NOT'
 'NOT' 'OFF' 'NOT' 'OFF' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT'
 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT'
 'OFF' 'NOT' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT'
 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'NOT' 'NOT'
 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'NOT'
 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'NOT' 'OFF' 'OFF' 'NOT'
 'OFF' 'NOT' 'OFF' 'OFF' 'NOT' 'NOT' 'NOT' 'NOT' 'N